In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance

#pip install plotly==5.8.0
#pip install geopy

In [ ]:
def get_data(base, url, size = 0):
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={d1053f2e2108904137d9b5d2753af8fe25bec4b00e73f7b9bec821dc}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)

        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names

        next_url = None

        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']

        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [ ]:
#Extract development-activity-monitor data
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'development-activity-monitor'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
DA_Monitor = data.copy()
#Rename Longitude and Latitude columns
DA_Monitor = DA_Monitor.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Completed:', DA_Monitor.shape[0], 'rows,', DA_Monitor.shape[1], 'columns extracted' )

Completed: 1403 rows, 43 columns extracted


In [ ]:
DA_Monitor

,data_format,development_key,status,year_completed,clue_small_area,clue_block,street_address,property_id,property_id_2,property_id_3,...,recreation_flr,publicdispaly_flr,community_flr,car_spaces,bike_spaces,town_planning_application,longitude,latitude,geopoint.lon,geopoint.lat
0,Pre May 16,X000288,COMPLETED,2011,Melbourne (CBD),104,150-154 A'Beckett Street MELBOURNE VIC 3000,100163,None,None,...,0,0,0,0,0,0,144.958971,-37.809597,144.958971,-37.809597
1,Pre May 16,X000449,COMPLETED,2009,North Melbourne,315,North Melbourne Recreation Reserve 204-206 Ard...,100514,None,None,...,2494,0,0,0,0,0,144.941433,-37.798926,144.941433,-37.798926
2,Pre May 16,X000395,COMPLETED,2005,Carlton,257,24-30 Barkly Place CARLTON VIC 3053,100732,None,None,...,0,0,0,0,0,0,144.961448,-37.803003,144.961448,-37.803003
3,Pre May 16,X000279,COMPLETED,2014,West Melbourne (Residential),101,49-53 Batman Street WEST MELBOURNE VIC 3003,100850,None,None,...,0,0,0,0,90,0,144.951554,-37.811134,144.951554,-37.811134
4,Pre May 16,X000231,COMPLETED,2003,Melbourne (CBD),87,11-21 Bennetts Lane MELBOURNE VIC 3000,101021,None,None,...,0,0,0,0,0,0,144.967682,-37.809166,144.967682,-37.809166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,Post Oct 2022,X0014203,UNDER CONSTRUCTION,None,Melbourne (Remainder),720,409-413 St Kilda Road MELBOURNE VIC 3004,108626,None,None,...,0,0,0,149,97,TP-2017-812/B,144.976861,-37.837365,144.976861,-37.837365
1399,Post Oct 2022,X0014704,UNDER CONSTRUCTION,None,Melbourne (CBD),44,423-447 Bourke Street MELBOURNE VIC 3000,101151,108109,108110,...,0,0,0,116,554,TPMR-2019-1/C,144.961200,-37.815100,144.961200,-37.815100
1400,Post Oct 22,X0014100,UNDER CONSTRUCTION,None,Port Melbourne,786,"130-188 Todd Road, Port Melbourne, 3207",692421,None,None,...,0,0,0,82,0,TP-2022-706,144.913305,-37.827508,144.913305,-37.827508
1401,Post May 16,X0011801,UNDER CONSTRUCTION,None,Carlton,245,86-94 Pelham Street CARLTON VIC 3053,107554,None,None,...,0,0,0,56,137,TP-2015-742/C,144.964800,-37.802300,144.964800,-37.802300
